In [1]:
import sys
sys.path.append("/home/khaymonenko/aste")

In [2]:
import pytorch_lightning as pl
from tqdm import tqdm
import torch
from transformers import AutoTokenizer, T5ForConditionalGeneration

from aste.data.readers import BankDataReader
from aste.train.data_providers import ModelDataset, ModelDatasetModule, ModelDatasetConfig
from aste.train.models import T5Model

from torch.multiprocessing import set_start_method
# set_start_method("spawn")

/home/khaymonenko/fairseq_virtualenv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
class Config:
    TRAIN_PATH = "/home/khaymonenko/aste/data/bank_3200/train_full.txt"
    DEV_PATH = "/home/khaymonenko/aste/data/bank_3200/dev_full.txt"
    TEST_PATH = "/home/khaymonenko/aste/data/bank_3200/test_full.txt"

    # MODEL_NAME = "ai-forever/FRED-T5-1.7B"
    MODEL_NAME = "t5-small"

    INPUT_MAX_LEN = 24
    OUTPUT_MAX_LEN = 24

    TRAIN_BATCH_SIZE = 1
    DEV_BATCH_SIZE = 1

    NUM_WORKERS = 1

    EPOCHS = 5

In [4]:
train_data = BankDataReader.from_file(Config.TRAIN_PATH)
val_data = BankDataReader.from_file(Config.DEV_PATH)

In [5]:
module_config = ModelDatasetConfig(
    tokenizer=AutoTokenizer.from_pretrained(Config.MODEL_NAME),
    train_batch_size=Config.TRAIN_BATCH_SIZE,
    dev_batch_size=Config.DEV_BATCH_SIZE,
    num_workers=Config.NUM_WORKERS,
    source_max_len=Config.INPUT_MAX_LEN,
    target_max_len=Config.OUTPUT_MAX_LEN,
)

dataset_module = ModelDatasetModule(train_data, val_data, module_config)

In [6]:
batch = next(iter(dataset_module.train_dataloader()))

In [7]:
model = T5Model(Config.MODEL_NAME)

# checkpoint_callback  = ModelCheckpoint(
#     dirpath="./",
#     filename="best_checkpoint",
#     save_top_k=2,
#     verbose=True,
#     monitor="val_loss",
#     mode="min"
# )

trainer = pl.Trainer(
    # callbacks = checkpoint_callback,
    max_epochs=Config.EPOCHS,
    devices=2,
    accelerator="gpu"
)

# trainer.fit(model, dataset_module.train_dataloader())

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [8]:
trainer.fit(model, dataset_module.val_dataloader())

/home/khaymonenko/fairseq_virtualenv/lib/python3.10/site-packages/pytorch_lightning/trainer/configuration_validator.py:74: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
Initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/2
/home/khaymonenko/fairseq_virtualenv/lib/python3.10/site-packages/pytorch_lightning/trainer/configuration_validator.py:74: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
Initializing distributed: GLOBAL_RANK: 1, MEMBER: 2/2
----------------------------------------------------------------------------------------------------
distributed_backend=nccl
All distributed processes registered. Starting with 2 processes
----------------------------------------------------------------------------------------------------



ProcessExitedException: process 0 terminated with signal SIGSEGV

In [9]:
batch

{'source_ids': tensor([[    3,     2,     3,     2,  6609,  6588,     2,  5345,     2,     3,
           6652, 14601, 22123,  9592,  6469,     3,     2,     3,     2, 18737,
          22036,  6588,     2,     1]]),
 'source_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]),
 'target_ids': tensor([[24697,  6725, 23912,  5814, 14391, 12377,  6609,   117,     3,     2,
           2044,     2, 17148,     2, 17238,  2533, 14709,  6725,     2,   117,
              3, 16034,     2,     1]])}

In [9]:
model.validation_step(batch, 0)

/home/khaymonenko/fairseq_virtualenv/lib/python3.10/site-packages/pytorch_lightning/core/module.py:423: You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet. This is most likely because the model hasn't been passed to the `Trainer`


tensor(4.2915)

In [10]:
import torch

In [12]:
torch.optim.Adam(model.parameters(), lr=0.0001)

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    eps: 1e-08
    foreach: None
    lr: 0.0001
    maximize: False
    weight_decay: 0
)